In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [5]:
df = loader.data

In [6]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'categorical', 'na_percentage': 0.0},
                'pir': {'type': 'categorical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [7]:
md1 = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [8]:
pm = PETsARD.HyperProcessor(metadata)

In [9]:
from PETsARD.Processor.Missingist import Missingist_Mean

In [10]:
config_1 = {'missingist': {'gen': Missingist_Mean(),
  'age': Missingist_Mean()},
 'outlierist': {'gen': None,
  'age': None},
 'encoder': {'gen': None,
  'age': None},
 'scaler': {'gen': None,
  'age': None}}

In [11]:
pm.fit(df, ['missingist', 'encoder', 'scaler'])

Mediator_Missingist is created.


In [12]:
df_transformed = pm.transform(df)

In [13]:
df_transformed

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,0.552374,0.645459,0.174722,0.402918,0.485938,-0.202041,0.264779,0.175474,0.0,0.0,0.365735,0.928549
1,0.510076,0.142249,0.319828,0.564424,0.778056,0.235906,0.002589,0.814753,0.0,0.0,0.106013,0.479589
2,0.772375,1.540055,0.319629,0.657013,0.092827,-0.056059,0.282363,0.360910,0.0,0.0,0.847481,0.925672
3,0.441507,0.309986,0.228349,0.525407,0.821072,1.929305,0.994960,0.205467,0.0,0.0,0.775311,0.611928
4,0.398028,-0.472785,0.361766,0.106693,0.735225,-1.296911,0.922416,0.581102,0.0,0.0,0.536615,0.421961
...,...,...,...,...,...,...,...,...,...,...,...,...
4185,0.748723,-1.423293,0.802125,0.468581,0.612218,-1.194723,0.680127,0.491498,0.0,0.0,0.235793,0.347592
4186,0.342176,1.428230,0.254055,0.595474,0.939544,-1.121732,0.881566,0.172599,0.0,0.0,0.014862,0.283087
4187,0.162816,1.651879,0.027710,0.278691,0.906043,0.265103,0.524414,0.389315,0.0,0.0,0.153136,0.328038
4188,0.860207,-0.864171,0.080072,0.801371,0.039815,-0.464810,0.639192,0.985955,0.0,0.0,0.149654,0.633602


In [14]:
df

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,62.0,White,Graduate,Married,27.799999,0,0,0.0,0.0,Q2,1
1,Male,53.0,White,HighSchool,Divorced,30.799999,0,1,0.0,0.0,Q1,0
2,Male,78.0,White,HighSchool,Married,28.799999,0,0,0.0,0.0,Q3,1
3,Female,56.0,White,Graduate,Parther,42.400002,1,0,0.0,0.0,Q3,0
4,Female,42.0,Black,College,Divorced,20.299999,1,0,0.0,0.0,Q4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4185,Male,25.0,Other,Graduate,Never,21.000000,0,0,0.0,0.0,Q1,0
4186,Female,76.0,White,HighSchool,Widowed,21.500000,1,0,0.0,0.0,Q1,0
4187,Female,80.0,White,College,Widowed,31.000000,0,0,0.0,0.0,Q1,0
4188,Male,35.0,White,9th,Married,26.000000,0,1,0.0,0.0,Q1,0


In [15]:
df_inverse = pm.inverse_transform(df_transformed)

In [19]:
(df_inverse == df).all()

gen      True
age     False
race     True
edu      True
mar      True
bmi     False
dep      True
pir      True
gh       True
mets     True
qm       True
dia      True
dtype: bool

In [23]:
df_inverse.loc[df_inverse['bmi'] != df['bmi'], 'bmi']

313     56.699997
856     59.099995
1222    58.799995
1624    14.499999
1767    14.499999
2343    57.199997
2703    56.699997
2930    57.999996
Name: bmi, dtype: float32

In [24]:
df.loc[df_inverse['bmi'] != df['bmi'], 'bmi']

313     56.700001
856     59.099998
1222    58.799999
1624    14.500000
1767    14.500000
2343    57.200001
2703    56.700001
2930    58.000000
Name: bmi, dtype: float32

In [25]:
pm.get_config(['age', 'bmi'])

{'missingist': {'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x10620e200>,
  'bmi': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1511e3d90>},
 'outlierist': {'age': <PETsARD.Processor.Outlierist.Outlierist_IQR at 0x1511e2a40>,
  'bmi': <PETsARD.Processor.Outlierist.Outlierist_IQR at 0x1511e2b60>},
 'encoder': {'age': None, 'bmi': None},
 'scaler': {'age': <PETsARD.Processor.Scaler.Scaler_Standard at 0x1511e1f60>,
  'bmi': <PETsARD.Processor.Scaler.Scaler_Standard at 0x1511e2260>}}